# LA Building Permits: 2013-2019

In [45]:
import pandas as pd
import matplotlib as mpl
import geopandas as gpd
import descartes
import geojson
import json
import altair as alt
import altair_latimes as lat
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context  
alt.renderers.enable('notebook')
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,}'.format

### Read data (as of 9/9/2019), clean up column headers

In [81]:
#https://data.lacity.org/A-Prosperous-City/Building-and-Safety-Permit-Information/yv23-pmwf
permits = pd.read_csv('/Users/mhustiles/Desktop/data/LA/Building_and_Safety_Permit_Information.csv')
permits.columns = permits.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

### Clean up data types

In [172]:
permits['project_number'] = permits['project_number'].astype(str)
permits['address_start'] = permits['address_start'].astype(str)
permits['address_end'] = permits['address_end'].astype(str)
permits['zip_code'] = permits['zip_code'].astype(str).replace('\.0', '', regex=True)
permits['assessor_book'] = permits['assessor_book'].astype(str).replace('\.0', '', regex=True)
permits['assessor_page'] = permits['assessor_page'].astype(str).replace('\.0', '', regex=True)
permits['license_#'] = permits['license_#'].astype(str)
permits['census_tract'] = permits['census_tract'].astype(str)
permits['council_district'] = permits['council_district'].astype(str)
permits['existing_code'] = permits['existing_code'].astype(str)
permits['proposed_code'] = permits['proposed_code'].astype(str)
permits['issue_date'] = pd.to_datetime(permits['issue_date'], format='%m/%d/%Y')
permits['status_date'] = pd.to_datetime(permits['status_date'], format='%m/%d/%Y')
permits['year-issued'] = permits['issue_date'].dt.year.astype(str)
permits['month-issued'] = permits['issue_date'].dt.month.astype(str)
permits['year-issued'] = permits['issue_date'].dt.year.astype(str)
permits['weekday-issued'] = permits['issue_date'].dt.weekday_name.astype(str)
permits['year-status'] = permits['status_date'].dt.year.astype(str)
permits['month-status'] = permits['status_date'].dt.month.astype(str)
permits['year-status'] = permits['status_date'].dt.year.astype(str)
permits['weekday-status'] = permits['status_date'].dt.weekday_name.astype(str)

### How many permits total?

In [173]:
# how many? 
len(permits)

1037228

### Filter list of all permits to just new construction on homes

In [174]:
new_home_permits = permits[(permits['permit_type'] == 'Bldg-New') & (permits['permit_sub-type'] == '1 or 2 Family Dwelling')]

In [175]:
#how many is that? 
len(new_home_permits)

21354

### Filter list of all permits to our zip codes

In [176]:
ourpermits = permits[permits['zip_code'].isin(['90169', '90210', '90077', '90049', '90272'])]

In [177]:
ourpermits.head()

,assessor_book,assessor_page,assessor_parcel,tract,block,lot,reference_#_old_permit_#,pcis_permit_#,status,status_date,...,latitude/longitude,applicant_relationship,existing_code,proposed_code,year-issued,month-issued,weekday-issued,year-status,month-status,weekday-status
23,4414,13,020,TR 9300,131,20,NaN,17042-90000-25965,Issued,2017-10-20,...,"(34.04671, -118.53835)",Net Applicant,nan,nan,2017,10,Friday,2017,10,Friday
55,4406,3,001,BRENTWOOD PARK,31,46,NaN,19042-10000-20034,Issued,2019-08-15,...,"(34.06265, -118.49062)",Agent for Contractor,nan,nan,2019,8,Thursday,2019,8,Thursday
132,4490,26,***,TR 31292-C,NaN,4,NaN,14042-20000-01021,Permit Finaled,2014-02-11,...,"(34.10923, -118.49495)",Contractor,nan,nan,2014,1,Friday,2014,2,Tuesday
195,4420,13,013,TR 17303,NaN,61,NaN,19042-30000-20090,Issued,2019-08-16,...,"(34.05164, -118.54004)",Owner-Bldr,nan,nan,2019,8,Friday,2019,8,Friday
204,4420,13,013,TR 17303,NaN,61,NaN,19041-30000-32091,Issued,2019-08-16,...,"(34.05164, -118.54004)",Owner-Bldr,nan,nan,2019,8,Friday,2019,8,Friday


### Basic descriptives about our permits

In [178]:
ourpermits.describe(include=['number']).round()

,valuation,floor_area-l.a._zoning_code_definition,#_of_residential_dwelling_units,#_of_accessory_dwelling_units,#_of_stories,floor_area-l.a._building_code_definition
count,"28,645.0","6,028.0","3,713.0",123.0,"5,811.0","41,867.0"
mean,"111,682.0","3,638.0",0.0,1.0,2.0,287.0
std,"678,913.0","7,758.0",3.0,0.0,61.0,"3,076.0"
min,0.0,"-2,998.0",-31.0,-3.0,-2.0,"-32,685.0"
25%,852.0,67.0,0.0,1.0,0.0,0.0
50%,"10,000.0","1,230.0",0.0,1.0,1.0,0.0
75%,"35,000.0","4,674.0",1.0,1.0,2.0,0.0
max,"33,750,000.0","204,600.0",81.0,2.0,"4,654.0","169,920.0"


In [179]:
ourpermits.columns

Index(['assessor_book', 'assessor_page', 'assessor_parcel', 'tract', 'block',
       'lot', 'reference_#_old_permit_#', 'pcis_permit_#', 'status',
       'status_date', 'permit_type', 'permit_sub-type', 'permit_category',
       'project_number', 'event_code', 'initiating_office', 'issue_date',
       'address_start', 'address_fraction_start', 'address_end',
       'address_fraction_end', 'street_direction', 'street_name',
       'street_suffix', 'suffix_direction', 'unit_range_start',
       'unit_range_end', 'zip_code', 'work_description', 'valuation',
       'floor_area-l.a._zoning_code_definition',
       '#_of_residential_dwelling_units', '#_of_accessory_dwelling_units',
       '#_of_stories', 'contractor's_business_name', 'contractor_address',
       'contractor_city', 'contractor_state', 'license_type', 'license_#',
       'principal_first_name', 'principal_middle_name', 'principal_last_name',
       'license_expiration_date', 'applicant_first_name',
       'applicant_last_name'

In [180]:
permit_types = ourpermits.groupby(['permit_type']).agg('size').sort_values(ascending=False).reset_index()
permit_sub_types = ourpermits.groupby(['permit_sub-type']).agg('size').sort_values(ascending=False).reset_index()

In [181]:
permit_types

,permit_type,0
0,Bldg-Alter/Repair,11956
1,Electrical,9532
2,Plumbing,8865
3,Grading,4374
4,HVAC,3351
5,Nonbldg-New,2757
6,Bldg-Addition,2688
7,Swimming-Pool/Spa,2623
8,Fire Sprinkler,2153
9,Bldg-New,1776


In [182]:
permit_sub_types

,permit_sub-type,0
0,1 or 2 Family Dwelling,42177
1,Apartment,5860
2,Commercial,4493
3,Onsite,209
4,Special Equipment,189
5,Public Safety Only,16
6,Offsite,2


### How many have been issued?

In [183]:
zip_count = ourpermits['year-issued'].groupby(ourpermits['zip_code'])
year_count = ourpermits['zip_code'].groupby(ourpermits['year-issued'])
license_count = ourpermits['zip_code'].groupby(ourpermits['license_type'])

In [171]:
year_count.count()

year-issued
2013    7189
2014    7334
2015    7773
2016    7807
2017    8277
2018    8782
2019    5784
Name: zip_code, dtype: int64

In [125]:
zip_count.count()

zip_code
90049    22378
90077     7923
90210     7138
90272    15507
Name: year-issued, dtype: int64

In [126]:
license_count.count()

license_type
3B         1
4          5
6          1
A        221
B      21660
C-4       17
C-5        1
C-6        3
C-7       99
C-8       35
C-9        1
C10     5775
C11      814
C12       21
C13        9
C15        4
C16     2126
C17       47
C20     2567
C21      152
C23        2
C27      146
C29       71
C33        4
C34        1
C35       15
C36     6142
C38       15
C39     1242
C42       39
C45      186
C46      350
C51        2
C53     2560
C54        4
C55        4
D03       31
D21        2
D24        1
D28        3
D30        1
D31       13
D34       38
D35       11
D40        1
D42        2
D51        1
D52        2
D62        2
D63        1
D64        1
PR         1
b         22
c10        1
c53        1
Name: zip_code, dtype: int64